In [2]:
import math
import pickle
import subprocess
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
from UDFManager import *
from pyudf.rotation import Quaternion

plt.style.use('presentation_')

In [3]:
# def quaternionToRotationArray(q):
#     return [
#         [1 - 2 * (q.q2 * q.q2 + q.q3 * q.q3), 2 * (q.q1 * q.q2 + q.q0 * q.q3), 2 * (q.q3 * q.q1 - q.q0 * q.q2)],
#         [2 * (q.q1 * q.q2 - q.q0 * q.q3), 1 - 2 * (q.q1 * q.q1 + q.q3 * q.q3), 2 * (q.q2 * q.q3 + q.q0 * q.q1)],
#         [2 * (q.q3 * q.q1 + q.q0 * q.q2), 2 * (q.q2 * q.q3 - q.q0 * q.q1), 1 - 2 * (q.q1 * q.q1 - q.q2 * q.q2)]
#     ]

# for j in range(4):
#     for i in np.linspace(20, 60, 5, dtype=np.uint8):
#         uobj = UDFManager(f'{j:0=2.0f}/udf/{i:0=3.0f}/output.udf')
#         nt = uobj.totalRecord()
#         start_pos = []
#         end_pos = []
#         for n in range(nt):
#             uobj.jump(n)
#             s = np.array(uobj.get("Particles[].R")[0])
#             e = s + np.array(quaternionToRotationArray(Quaternion(*uobj.get("Particles[].q")[0]))[1])
#             start_pos.append(s)
#             end_pos.append(e)
#         start_pos = np.array(start_pos)
#         end_pos = np.array(end_pos)
        
#         np.save(f'positions/{j:0=2.0f}_{i:0=3.0f}_start', start_pos)
#         np.save(f'positions/{j:0=2.0f}_{i:0=3.0f}_end', end_pos)

# for j in range(4):
#     for i in np.linspace(20, 60, 5, dtype=np.uint8):
#         start_pos = np.load(f'positions/{j:0=2.0f}_{i:0=3.0f}_start.npy')
#         end_pos = np.load(f'positions/{j:0=2.0f}_{i:0=3.0f}_end.npy')

In [4]:
EPS = 12
for j in range(4):
    for i in np.linspace(20, 60, 5, dtype=np.uint8):
        start_pos = np.load(f'positions/{j:0=2.0f}_{i:0=3.0f}_start.npy')
        end_pos = np.load(f'positions/{j:0=2.0f}_{i:0=3.0f}_end.npy')

        for n in tqdm(range(401)):
            start_position = [start_pos[n][0], start_pos[n][1]]
            end_position = [(end_pos[n][0] - start_pos[n][0]) * EPS, (end_pos[n][1] - start_pos[n][1]) * EPS]
            fig, ax = plt.subplots(figsize=(4, 4))
            ax.set_xlim([0, 64])
            ax.set_ylim([32, 96])
            ax.tick_params(top=False, bottom=False, right=False, left=False)
            ax.set_xticklabels(labels=[])
            ax.set_yticklabels(labels=[])
            particle = patches.Circle(
                tuple(start_position),
                radius=5,
                fc='w',
                linewidth=3
            )
            arrow = patches.FancyArrow(
                *start_position,
                *end_position,
                width=2,
                head_width=4,
                head_length=3,
                ec='k',
                fc='k'
            )
            ax.add_patch(arrow)
            ax.add_patch(particle)
            plt.savefig(f'../figs/sim07/figs4movies/{j:0=2.0f}/{i:0=3.0f}/_{n:0=3.0f}.png')
            plt.clf()
            plt.close()

In [76]:
# シミュレーションの模式図用。

EPS = 25
for j in range(4):
    for i in np.linspace(20, 60, 5, dtype=np.uint8):
        start_pos = np.load(f'positions/{j:0=2.0f}_{i:0=3.0f}_start.npy')
        end_pos = np.load(f'positions/{j:0=2.0f}_{i:0=3.0f}_end.npy')

        n = 400

        gap = (start_pos[n][0] - 32, start_pos[n][1] - 64)

        start_position = [start_pos[n][0] - gap[0], start_pos[n][1] - gap[1]]
        end_position = [((end_pos[n][0] - gap[0]) - start_position[0]) * EPS, ((end_pos[n][1] - gap[1]) - start_position[1]) * EPS]

        fig, ax = plt.subplots(figsize=(4, 4))
        ax.set_xlim([0, 64])
        ax.set_ylim([32, 96])
        ax.tick_params(top=False, bottom=False, right=False, left=False)
        ax.set_xticklabels(labels=[])
        ax.set_yticklabels(labels=[])
        particle = patches.Circle(
            tuple(start_position),
            radius=8,
            fc='w',
            linewidth=3
        )
        arrow = patches.FancyArrow(
            *start_position,
            *end_position,
            width=4,
            head_width=8,
            head_length=3,
            ec='k',
            fc='k'
        )
        if (j == 0 and i < 60) or (j == 1 and i < 50) or (j == 2 and i < 40):
            ax.set_facecolor("#B4C7E7")
            ax.add_patch(arrow)
        else:
            ax.set_facecolor("#F8CBAD")
        ax.add_patch(particle)
        plt.savefig(f'../figs/sim07/figs4movies/{j:0=2.0f}/{i:0=3.0f}/fixed.png')
        plt.clf()
        plt.close()

In [5]:
# 理論値との比較用。単体で動画を作成する。

EPS = 12
for j in range(1):
    for i in np.linspace(20, 60, 5, dtype=np.uint8):
        subprocess.run(['ffmpeg', '-f', 'image2', '-r', '15', '-i', f'../figs/sim07/figs4movies/{j:0=2.0f}/{i:0=3.0f}/_%03d.png', '-r', '15', '-an', '-vcodec', 'libx264', '-pix_fmt', 'yuv420p', f'../figs/sim07/_{j:0=2.0f}_{i:0=3.0f}_video.mp4'])
        subprocess.run(['ffmpeg', '-i', f'../figs/sim07/_{j:0=2.0f}_{i:0=3.0f}_video.mp4', f'../figs/sim07/_{j:0=2.0f}_{i:0=3.0f}_video.gif'])